This notebook process raw Khmer Alphabet images, into 28x28 pixels, to then train the Convolutional Neural Network to classify 33 Khmer alphabets.

First process raw 3300 Khmer Alphabet. 

In [1]:
import numpy as np
import pandas as pd
from PIL import Image


## Import Images and Process
handwriting_dir = "./dataset"

csv_file_path = handwriting_dir+'/label.csv'
data = pd.read_csv(csv_file_path)

processed_images = []
labels = []

for image_file, label in zip(data['image_file'], data['label']):
    try:
        image = Image.open(handwriting_dir+"/"+image_file)
        image = image.convert('L')
        image = image.resize((28, 28))  # Resize to 28x28
        image_array = np.array(image) / 255.0  # Normalize to [0, 1]
        processed_images.append(image_array)
        labels.append(label)
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

# Convert to numpy arrays
X = np.array(processed_images)
y = np.array(labels)



Encode the 33 alphabets into uniques number (0-->31) 
Example: ក (kor) is encoded into 0, ខ (khor) is encoded into 1, and so on until the last alphabet. 

In [2]:
from sklearn.preprocessing import LabelEncoder

# Encode the 33 labels
label_encoder = LabelEncoder()
label_encoder.fit(y)
y_numeric = label_encoder.transform(y)
unique_labels_num = set(y_numeric)

Split the dataset into training (80%), validation (10%), and test (10%), dataset.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y_numeric, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Set up with CNN model with multiple hidden layers, and drop out layers. Drop out layers are important to prevent overfitting. 

In [4]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Define the CNN
model = Sequential()

model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# 64 convolution filters used each of size 3x3
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# choose the best features via pooling
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# randomly turn neurons on and off to improve convergence
model.add(layers.Dropout(0.25))
# flatten since too many dimensions, we only want a classification output
model.add(layers.Flatten())
# fully connected to get all relevant data
model.add(layers.Dense(128, activation='relu'))
# one more dropout
model.add(layers.Dropout(0.5))
# output a softmax to squash the matrix into output probabilities
model.add(layers.Dense(33, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

/opt/miniconda3/envs/cropapp/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
epochs = 20
batch_size = 32
# Train the model
model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=batch_size,
         validation_data=(X_val, y_val))  # Use the validation set

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

Epoch 1/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0505 - loss: 3.4573 - val_accuracy: 0.4576 - val_loss: 2.5950
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3227 - loss: 2.3883 - val_accuracy: 0.7576 - val_loss: 1.1250
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5815 - loss: 1.3759 - val_accuracy: 0.8364 - val_loss: 0.6662
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7159 - loss: 0.9574 - val_accuracy: 0.8606 - val_loss: 0.4815
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7667 - loss: 0.7603 - val_accuracy: 0.9000 - val_loss: 0.3767
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8188 - loss: 0.6030 - val_accuracy: 0.9333 - val_loss: 0.2946
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8226 - loss: 0.5891 - val_accuracy: 0.9152 - val_loss: 0.2797
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8598 - loss: 0.4399 - val_accuracy: 0.9242 - val_loss:

Save the model into local machine for later use

In [6]:
import joblib # export label 
model.save('my_model.h5')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']